<a href="https://colab.research.google.com/github/cagBRT/what-if-tool/blob/main/4_What_If_Tool_with_Keras_and_Scikit_Learn_Cloud_AI_Platform_Models_end_to_end.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

modify for using colab only
must use tpus

# Comparing Keras and Scikit models deployed on Cloud AI Platform with the What-if Tool

In this notebook we'll use the UCI [wine quality dataset](https://archive.ics.uci.edu/ml/datasets/wine+quality) to train both tf.keras and Scikit learn regression models that will predict the quality rating of a wine given 11 numerical data points about the wine. You'll learn how to:
* Build, train, and then deploy tf.keras and Scikit Learn models to Cloud AI Platform
* Use the What-if Tool to compare two different models deployed on CAIP

You will need a Google Cloud Platform account and project to run this notebook. Instructions for creating a project can be found [here](https://cloud.google.com/resource-manager/docs/creating-managing-projects).

## Installing dependencies

In [7]:
# If running in colab then pip install, otherwise no need.
try:
  import google.colab
  !pip install --upgrade tensorflow>=2.0.0 witwidget 
except Exception:
  pass

In [8]:
import sys
python_version = sys.version_info[0]

In [9]:
# If you're running on Colab, you'll need to install the What-if Tool package and authenticate
def pip_install(module):
    if python_version == '2':
        !pip install {module} --quiet
    else:
        !pip3 install {module} --quiet

try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    pip_install('witwidget')

    from google.colab import auth
    auth.authenticate_user()

In [10]:
import pandas as pd
import numpy as np
import tensorflow as tf
import witwidget
import os
import pickle

from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

from sklearn.utils import shuffle
from sklearn.linear_model import LinearRegression
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder

# This has been tested on TF 1.14
print(tf.__version__)

2.5.0


## Download and process data

In this section we'll:
* Download the wine quality data directly from UCI Machine Learning
* Read it into a Pandas dataframe and preview it
* Split the data and labels into train and test sets

In [11]:
!wget 'http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv'

--2021-07-28 23:06:38--  http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 264426 (258K) [application/x-httpd-php]
Saving to: ‘winequality-white.csv’

winequality-white.c 100%[===================>] 258.23K  1.16MB/s    in 0.2s    

2021-07-28 23:06:39 (1.16 MB/s) - ‘winequality-white.csv’ saved [264426/264426]



In [12]:
data = pd.read_csv('winequality-white.csv', index_col=False, delimiter=';')
data = shuffle(data, random_state=4)

In [13]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
3545,7.3,0.20,0.29,19.50,0.039,69.0,237.0,1.00037,3.10,0.48,9.2,6
2460,6.6,0.33,0.24,16.05,0.045,31.0,147.0,0.99822,3.08,0.52,9.2,5
3855,5.9,0.18,0.29,4.60,0.032,68.0,137.0,0.99159,3.21,0.38,11.3,6
3427,5.8,0.24,0.26,10.05,0.039,63.0,162.0,0.99375,3.33,0.50,11.2,6
2033,7.2,0.23,0.33,12.70,0.049,50.0,183.0,0.99870,3.41,0.40,9.8,5


In [14]:
labels = data['quality']

In [15]:
print(labels.value_counts())

6    2198
5    1457
7     880
8     175
4     163
3      20
9       5
Name: quality, dtype: int64


In [16]:
data = data.drop(columns=['quality'])

In [17]:
train_size = int(len(data) * 0.8)
train_data = data[:train_size]
train_labels = labels[:train_size]

test_data = data[train_size:]
test_labels = labels[train_size:]

In [18]:
train_data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
3545,7.3,0.20,0.29,19.50,0.039,69.0,237.0,1.00037,3.10,0.48,9.2
2460,6.6,0.33,0.24,16.05,0.045,31.0,147.0,0.99822,3.08,0.52,9.2
3855,5.9,0.18,0.29,4.60,0.032,68.0,137.0,0.99159,3.21,0.38,11.3
3427,5.8,0.24,0.26,10.05,0.039,63.0,162.0,0.99375,3.33,0.50,11.2
2033,7.2,0.23,0.33,12.70,0.049,50.0,183.0,0.99870,3.41,0.40,9.8


## Train tf.keras model

In this section we'll:

* Build a regression model using tf.keras to predict a wine's quality score
* Train the model
* Add a layer to the model to prepare it for serving

In [19]:
# This is the size of the array we'll be feeding into our model for each wine example
input_size = len(train_data.iloc[0])
print(input_size)

11


In [20]:
model = Sequential()
model.add(Dense(200, input_shape=(input_size,), activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(25, activation='relu'))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam')

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 200)               2400      
_________________________________________________________________
dense_1 (Dense)              (None, 50)                10050     
_________________________________________________________________
dense_2 (Dense)              (None, 25)                1275      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 26        
Total params: 13,751
Trainable params: 13,751
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.fit(train_data.values,train_labels.values, epochs=4, batch_size=32, validation_split=0.1)

Epoch 1/4
111/111 [==============================] - 1s 5ms/step - loss: 1.9201 - val_loss: 0.8386
Epoch 2/4
111/111 [==============================] - 1s 5ms/step - loss: 0.7882 - val_loss: 0.8541
Epoch 3/4
111/111 [==============================] - 0s 4ms/step - loss: 0.8383 - val_loss: 0.8159
Epoch 4/4
111/111 [==============================] - 0s 4ms/step - loss: 0.7045 - val_loss: 1.0029


## Deploy keras model to Cloud AI Platform

In this section we'll:
* Set up some global variables for our GCP project
* Add a serving layer to our model so we can deploy it on Cloud AI Platform
* Run the deploy command to deploy our model
* Generate a test prediction on our deployed model

In [23]:
# Update these to your own GCP project + model names
GCP_PROJECT = 'your_gcp_project'
KERAS_MODEL_BUCKET = 'gs://your_storage_bucket'
KERAS_VERSION_NAME = 'v1'

In [24]:
tf.config.experimental_connect_to_host('grpc://' + os.environ['COLAB_TPU_ADDR'])
resolver = tf.distribute.cluster_resolver.TPUClusterResolver('grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.experimental.TPUStrategy(resolver) 

INFO:tensorflow:Initializing the TPU system: grpc://10.53.192.122:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.53.192.122:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [25]:
# Add the serving input layer below in order to serve our model on AI Platform
class ServingInput(tf.keras.layers.Layer):
  # the important detail in this boilerplate code is "trainable=False"
  def __init__(self, name, dtype, batch_input_shape=None):
    super(ServingInput, self).__init__(trainable=False, name=name, dtype=dtype, batch_input_shape=batch_input_shape)
  def get_config(self):
    return {'batch_input_shape': self._batch_input_shape, 'dtype': self.dtype, 'name': self.name }

restored_model = model

serving_model = tf.keras.Sequential()
serving_model.add(ServingInput('serving', tf.float32, (None, input_size)))
serving_model.add(restored_model)
tf.contrib.saved_model.save_keras_model(serving_model, os.path.join(KERAS_MODEL_BUCKET, 'keras_export'))  # export the model to your GCS bucket
export_path = KERAS_MODEL_BUCKET + '/keras_export'

AttributeError: ignored

In [ ]:
# Configure gcloud to use your project
!gcloud config set project $GCP_PROJECT

In [ ]:
# Create a new model in our project, you only need to run this once
!gcloud ai-platform models create keras_wine

In [ ]:
# Deploy the model to Cloud AI Platform
!gcloud beta ai-platform versions create $KERAS_VERSION_NAME --model keras_wine \
--origin=$export_path \
--python-version=3.5 \
--runtime-version=1.14 \
--framework='TENSORFLOW'

In [ ]:
%%writefile predictions.json
[7.8, 0.21, 0.49, 1.2, 0.036, 20.0, 99.0, 0.99, 3.05, 0.28, 12.1]

In [ ]:
# Test the deployed model on an example from our test set
# The correct score for this prediction is 7
prediction = !gcloud ai-platform predict --model=keras_wine --json-instances=predictions.json --version=$KERAS_VERSION_NAME
print(prediction[1])

## Build and train Scikit learn model

In this section we'll:
* Train a regression model using Scikit Learn
* Save the model to a local file using `pickle`

In [ ]:
SKLEARN_VERSION_NAME = 'v1'
SKLEARN_MODEL_BUCKET = 'gs://sklearn_model_bucket'

In [ ]:
scikit_model = LinearRegression().fit(train_data.values, train_labels.values)

In [ ]:
# Export the model to a local file using pickle
pickle.dump(scikit_model, open('model.pkl', 'wb'))

## Deploy Scikit model to CAIP

In this section we'll:
* Copy our saved model file to Cloud Storage
* Run the gcloud command to deploy our model
* Generate a prediction on our deployed model

In [ ]:
# Copy the saved model to Cloud Storage
!gsutil cp ./model.pkl gs://wine_sklearn/model.pkl

In [ ]:
# Create a new model in our project, you only need to run this once
!gcloud ai-platform models create sklearn_wine

In [ ]:
!gcloud beta ai-platform versions create $SKLEARN_VERSION_NAME --model=sklearn_wine \
--origin=$SKLEARN_MODEL_BUCKET \
--runtime-version=1.14 \
--python-version=3.5 \
--framework='SCIKIT_LEARN'

In [ ]:
# Test the model usnig the same example instance from above
!gcloud ai-platform predict --model=sklearn_wine --json-instances=predictions.json --version=$SKLEARN_VERSION_NAME

## Compare tf.keras and Scikit models with the What-if Tool

Now we're ready for the What-if Tool! In this section we'll:
* Create an array of our test examples with their ground truth values. The What-if Tool works best when we send the actual values for each example input.
* Instantiate the What-if Tool using the `set_compare_ai_platform_model` method. This lets us compare 2 models deployed on Cloud AI Platform.

In [ ]:
# Create np array of test examples + their ground truth labels
test_examples = np.hstack((test_data[:200].values,test_labels[:200].values.reshape(-1,1)))
print(test_examples.shape)

In [ ]:
# Create a What-if Tool visualization, it may take a minute to load
# See the cell below this for exploration ideas

# We use `set_predict_output_tensor` here becuase our tf.keras model returns a dict with a 'sequential' key

config_builder = (WitConfigBuilder(test_examples.tolist(), data.columns.tolist() + ['quality'])
  .set_ai_platform_model(GCP_PROJECT, 'keras_wine', KERAS_VERSION_NAME).set_predict_output_tensor('sequential').set_uses_predict_api(True)
  .set_target_feature('quality')
  .set_model_type('regression')
  .set_compare_ai_platform_model(GCP_PROJECT, 'sklearn_wine', SKLEARN_VERSION_NAME))
WitWidget(config_builder, height=800)

## What-if Tool Exploration ideas

* Look at the scatter plot of "Inference value scikit_wine" vs "Inference value keras_wine"
  * Examples off of the diagonal represent wines for which the two models have large disagreement on the quality score. Click on some of these and explore the features. 
  * You can also click on individual examples, change some of the feature values for that example, and compare the impact of that change on the model's prediction
  * Check out the partial dependence plots to see what features are causes the large skew between the two models.

* Go to the Performance tab and see the overall performance of each model. Is one more accurate over the test data than the other?
  * In this tab, use the "Slice by" dropdown to slice the data into subgroups and see how both models perform across those subgroups. Try slicing by alcohol. Which model has more consistent performance across the slices?